Form Κ3-02-025


In [ ]:
ignore_forms = ["K6-04-001", "K3-02-023", "K3-02-029", "K2-02-035", "K2-02-031", "K2-02-030", "K3-02-027"]
notify = ["K1-03-111", "K3-03-005", "K1-03-114", "K1-03-116", "K1-03-113", "K3-02-031", "K3-02-025", "K5-01-003", "K5-01-005"]

In [ ]:
from selenium import webdriver

driver = webdriver.Safari()
driver.maximize_window()


driver.get("https://www.athexgroup.gr/web/guest/companies-information-corporate-actions")

psource = driver.page_source

In [ ]:
from bs4 import BeautifulSoup
import re, datetime, requests, os

def extract_hrefs(html_string):
    # Regex zum Extrahieren von href-Werten
    href_pattern = r'href=["\'](.*?)["\']'
    # Alle Übereinstimmungen finden
    hrefs = re.findall(href_pattern, html_string)
    return hrefs

def save_actions(psource:str):
    last_change_file = "/Users/a2/code/fin/trade/data/main_changes/company/AT/last_change_date.date"
    folder_corp_act = "/Users/a2/code/fin/trade/data/main_changes/company/AT"
    now= datetime.date.today().isoformat()
    with open(last_change_file, "r") as f:
        tstp =f.read()
    dt = datetime.date.fromisoformat(tstp)
    soup = BeautifulSoup(psource, "lxml")
    main_div = soup.select("#p_p_id_101_INSTANCE_E8Xk2zINhmZA_")[0]

    table = main_div.select("table")[0]

    rows = table.select("tr")
    for tr in rows:
        content = tr.select("a")[0]
        title = content.text
        pattern = re.compile("(\d\d) - (\d\d) - (\d\d\d\d)")
        mat = re.match(pattern, title)
        day = mat.group(1)
        month = mat.group(2)
        year = mat.group(3)
        article_date = datetime.date(year=year, month=month, day=day)
        if article_date < dt:
            with open(last_change_file, "w") as f:
                f.write(now)
            return
        link = content["href"]
        pdf_source = requests.get(link).text
        hrefs = extract_hrefs(pdf_source)
        for l in hrefs:
            if l.startswith("/documents"):
                if "Cookies" in l:
                    continue
                link_pdf = "https://www.athexgroup.gr" + l
                break
        else:
            link_pdf = ""
        if link_pdf != "":
            pdf = requests.get(link_pdf).content
            with open(os.path.join(folder_corp_act, title+".pdf"), "wb") as f:
                print("written "+title)
                f.write(pdf)
    with open(last_change_file, "w") as f:
        f.write(now)
            
        


In [ ]:
import datetime
last_change_file = "/Users/a2/code/fin/trade/data/main_changes/company/AT/last_change_date.date"

with open(last_change_file, "r") as f:
    tstp =f.read()

now= datetime.date.today().isoformat()

In [36]:
psource

UsageError: Missing object name.
